In [1]:
# from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
# from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
# from sklearn.preprocessing import scale

from scipy.spatial.distance import euclidean, cityblock
import numpy as np
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph

from music21 import *
from alfabeto_sources.all_sources import *
from alfabeto_code.AlfabetoConverter import AlfabetoDict
from Continuo.kmeans_inversion import k_means_data, inversion_data, silhouette_analysis, label_maker_alf
# from Continuo.kmeans import final_alf_labels
from alfabeto_data.pickled_data import *

In [2]:
temp_maj = {0: 0.184, 1: 0.001, 2: 0.155, 3: 0.003, 4: 0.191,
            5: 0.109, 6: 0.005, 7: 0.214, 8: 0.001, 9: 0.078,
            10: 0.004, 11: 0.055}

temp_min = {0: 0.192, 1: 0.005, 2: 0.149, 3: 0.179, 4: 0.002,
            5: 0.144, 6: 0.002, 7: 0.201, 8: 0.038, 9: 0.012,
            10: 0.053, 11: 0.022}

t_maj = [temp_maj[x] * 100 for x in range(12)]
t_min = [temp_min[x] * 100 for x in range(12)]

# pal_data = palestrina_notes_data
# pal_data[0].append(t_maj)
# pal_data[0].append(t_min)
# pal_data[1].append('TMAJ')
# pal_data[1].append('TMIN')

bach_data = bach_notes_data


alf_data = alfabeto_notes_data
alf_data[0].append(t_maj)
alf_data[0].append(t_min)
alf_data[1].append('TMAJ')
alf_data[1].append('TMIN')

tonal_fixer = {(0,0): 'CM', (0,9): 'am', (1, 7): 'GM', (1, 4): 'em', (2, 2): 'DM', (2, 11): 'bm', 
               (3, 9): 'AM', (3, 6): 'f♯m', (4, 4): 'EM', (4, 1): 'c♯m', (5, 11): 'BM', (5, 8): 'g♯m', 
               (6, 6): 'F♯M', (6, 3): 'd♯m', 
               (-1,5): 'FM', (-1,2): 'dm', (-2,10): 'B♭M', (-2,7): 'gm', (-3,3): 'E♭M', (-3,0): 'cm', 
               (-4,8): 'A♭M', (-4,5): 'fm', (-5,1): 'D♭M', (-5,10): 'b♭m', (-6,6): 'G♭M', (-6,3): 'e♭m'}

def label_maker(label_tuples):
    final_labels = []
    for x in label_tuples:
        if x in tonal_fixer.keys():
            final_labels.append('$'+tonal_fixer[x]+'$')
        elif x == 'TMAJ':
            final_labels.append('$M$')
        elif x == 'TMIN':
            final_labels.append('$m$')
        
    return final_labels

def label_maker_alfabeto(corpus_list):
    alf_labels = []
    for x in corpus_list: #do this if the tuples are in list of list
        if x == ('f',2) or x == (-1, 2):
            alf_labels.append('$♭:D$')
        elif x == ('n',4) or x == (0, 4):
            alf_labels.append('$♮:E$')
        elif x == ('f',4) or x == (-1, 4):
            alf_labels.append('$♭:E$')
        elif x == ('n',9) or x == (0, 9):
            alf_labels.append('$♮:A$')
        elif x == ('f',7) or x == (-1, 7):
            alf_labels.append('$♭:G$')
        elif x == ('n',2) or x == (0, 2):
            alf_labels.append('$♮:D$')
        elif x == ('f',9) or x == (-1, 9):
            alf_labels.append('$♭:A$')
        elif x == ('f',0) or x == (-1, 0):
            alf_labels.append('$♭:C$')
        elif x == ('f',10) or x == (-1, 10):
            alf_labels.append('$♭:B♭$')
        elif x == ('n',5) or x == (0, 5):
            alf_labels.append('$♮:F$')
        elif x == ('n',0) or x == (0, 0):
            alf_labels.append('$♮:C$')
        elif x == ('f',5) or x == (-1, 5):
            alf_labels.append('$♭:F$')
        elif x == ('n',7) or x == (0, 7):
            alf_labels.append('$♮:G$')
        elif x == 'TMAJ':
            alf_labels.append('$M$')
        elif x == 'TMIN':
            alf_labels.append('$m$')
        else:
            alf_labels.append('o')
            print(x)
    return alf_labels

def label_maker_alfabeto_labeled(corpus_list):
    alf_labels = []
    for x in corpus_list: #do this if the tuples are in list of list
        if x == ('f',2) or x == (-1, 2):
            alf_labels.append('$A ♭:D$')
        elif x == ('n',4) or x == (0, 4):
            alf_labels.append('$A ♮:E$')
        elif x == ('f',4) or x == (-1, 4):
            alf_labels.append('$A ♭:E$')
        elif x == ('n',9) or x == (0, 9):
            alf_labels.append('$A ♮:A$')
        elif x == ('f',7) or x == (-1, 7):
            alf_labels.append('$A ♭:G$')
        elif x == ('n',2) or x == (0, 2):
            alf_labels.append('$A ♮:D$')
        elif x == ('f',9) or x == (-1, 9):
            alf_labels.append('$A ♭:A$')
        elif x == ('f',0) or x == (-1, 0):
            alf_labels.append('$A ♭:C$')
        elif x == ('f',10) or x == (-1, 10):
            alf_labels.append('$A ♭:B♭$')
        elif x == ('n',5) or x == (0, 5):
            alf_labels.append('$A ♮:F$')
        elif x == ('n',0) or x == (0, 0):
            alf_labels.append('$A ♮:C$')
        elif x == ('f',5) or x == (-1, 5):
            alf_labels.append('$A ♭:F$')
        elif x == ('n',7) or x == (0, 7):
            alf_labels.append('$A ♮:G$')
        elif x == 'TMAJ':
            alf_labels.append('$M$')
        elif x == 'TMIN':
            alf_labels.append('$m$')
        else:
            alf_labels.append('o')
            print(x)
    return alf_labels

def label_maker_pal(corpus_list):
    alf_labels = []
    for x in corpus_list: #do this if the tuples are in list of list
        if x == ('f',2) or x == (-1, 2):
            alf_labels.append('$P ♭:D$')
        elif x == ('n',4) or x == (0, 4):
            alf_labels.append('$P ♮:E$')
        elif x == ('f',4) or x == (-1, 4):
            alf_labels.append('$P ♭:E$')
        elif x == ('n',9) or x == (0, 9):
            alf_labels.append('$P ♮:A$')
        elif x == ('f',7) or x == (-1, 7):
            alf_labels.append('$P ♭:G$')
        elif x == ('n',2) or x == (0, 2):
            alf_labels.append('$P ♮:D$')
        elif x == ('f',9) or x == (-1, 9):
            alf_labels.append('$P ♭:A$')
        elif x == ('f',0) or x == (-1, 0):
            alf_labels.append('$P ♭:C$')
        elif x == ('f',10) or x == (-1, 10):
            alf_labels.append('$P ♭:B♭$')
        elif x == ('n',5) or x == (0, 5):
            alf_labels.append('$P ♮:F$')
        elif x == ('n',0) or x == (0, 0):
            alf_labels.append('$P ♮:C$')
        elif x == ('f',5) or x == (-1, 5):
            alf_labels.append('$P ♭:F$')
        elif x == ('n',7) or x == (0, 7):
            alf_labels.append('$P ♮:G$')
        elif x == 'TMAJ':
            alf_labels.append('$M$')
        elif x == 'TMIN':
            alf_labels.append('$m$')
        else:
            alf_labels.append('o')
            print(x)
    return alf_labels

def label_maker_bach(label_tuples):
    final_labels = []
    for x in label_tuples:
        if x in tonal_fixer.keys():
#             print('$'+'B '+tonal_fixer[x]+'$')
            final_labels.append('$'+'B '+tonal_fixer[x]+'$')
        elif x == 'TMAJ':
            final_labels.append('$M$')
        elif x == 'TMIN':
            final_labels.append('$m$')
    return final_labels

In [3]:
def averaged_octave_harmonization(flattened_list_with_labels, label_function):
    total_chunks = []
    averaged_chunks = []
    all_keys = []
    final_unchunked = []
    for x in flattened_list_with_labels[1]:
        if x not in all_keys:
            all_keys.append(x)
    for x in all_keys:
        a_key = []
        for a, b in zip(flattened_list_with_labels[0], flattened_list_with_labels[1]):
            if b == x:
                a_key.append(a)
        averaged_chunks.append([sum(col) / len(col) for col in zip(*a_key)])
    
    return averaged_chunks, label_function(all_keys)

In [14]:
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 7)
alf_ave = averaged_octave_harmonization(alf_data, label_maker_alfabeto)
bach_ave = averaged_octave_harmonization(bach_data, label_maker_bach)
# pal_ave = averaged_octave_harmonization(pal_data, label_maker_pal)

# all_labels_all = []
# all_keys_all = []
# for x, y in zip(alf_ave[0], alf_ave[1]):
#     all_labels_all.append(x)
#     all_keys_all.append(y)
# for x, y in zip(pal_ave[0], pal_ave[1]):
#     all_labels_all.append(x)
#     all_keys_all.append(y)
# for x, y in zip(bach_ave[0], bach_ave[1]):
#     all_labels_all.append(x)
#     all_keys_all.append(y)

# all_all = (all_labels_all, all_keys_all)
# bach_ave
# k_means_data(alf_ave[0], 2, alf_ave[1])
# k_means_data(tester[0], 2, label_maker(tester[1]))

Populating the interactive namespace from numpy and matplotlib


In [17]:
all_all

([[18.483801790513255,
   0.99005594441991307,
   13.600365519264766,
   0.79941766113743074,
   11.447827545094759,
   12.020709849743952,
   1.5307780863528335,
   18.178525780683287,
   0.13350999795854154,
   13.057741587149325,
   4.6887762657180083,
   5.0684899719639152],
  [16.664998954684528,
   0.18118930947528442,
   11.287143968567015,
   11.939675301702243,
   2.6630513599422265,
   12.829080956092872,
   0.63031214448619388,
   17.811000778637247,
   5.5427407554436305,
   4.8361278200455331,
   12.652484336089492,
   2.9621943148337273],
  [16.952157105359873,
   0.83760378119916234,
   13.583216505074549,
   0.024884259259259259,
   13.88101949160129,
   9.1553400063875277,
   2.6104366469017832,
   18.310653440885016,
   0.82900038515713814,
   13.231713234792739,
   0.95234559390883267,
   9.6316295494728195],
  [17.880952482690894,
   1.4745132078309968,
   8.9548306322803697,
   12.031136474703681,
   2.7769756036197704,
   13.726729759018937,
   0.26352479817928909

In [29]:
def k_means_simple(list_of_lists, cluster_number, label_markers):
    np.random.seed(42)

    data = np.array(list_of_lists)
    label_dict = {}
    label_numbers = []
    n = 0
    label_dict[label_markers[0]] = 0
    for j in range(1, len(label_markers)):
        if label_markers[j] not in label_dict:
            label_dict[label_markers[j]] = n+1
            n += 1

    for i in label_markers:
        label_numbers.append(label_dict[i])
    n_samples, n_features = data.shape
    n_digits = cluster_number
    labels = np.array(label_numbers)
    # print('label numbers', label_numbers)
    # sample_size = 300

    sample_size = len(data)    

    k_data = KMeans(init='k-means++', n_clusters=n_digits, n_init=10).fit(data)
    silhouette = metrics.silhouette_score(data, k_data.labels_, metric='euclidean', sample_size=sample_size)
    completeness = metrics.completeness_score(labels, k_data.labels_)
#     pca = PCA(n_components=2).fit_transform(data)
    pca = PCA(n_components=2).fit(data).score(data)
    return {'silhouette': silhouette, 'completeness': completeness, 'pca': pca}

In [7]:
def neato_graph(list_of_lists, label_lists):
    dt = [('len', float)]

    A = (np.array(list_of_lists)).view(dt)

    G = nx.from_numpy_matrix(A)
    relabeled_nodes = {}
    for x, y in zip (range(len(label_lists)), label_lists):
        relabeled_nodes[x] = y[1:-1]
        print(y[1:-1])
    G = nx.relabel_nodes(G, relabeled_nodes)

    G = to_agraph(G)

    G.node_attr.update(style='filled', fontsize=50)
#     G.edge_attr.update(color="black", width=0.5)
    G.edge_attr.update(color='black', style='invis', width=0)
    # G.graph_attr.update(landscape='true', ranksep='0.1')
    #  ♭ ♮ ...for copy and paste
    
    modal_pairs = {'ionian':('♮:C','♭:F'), 'dorian':('♭:G', '♮:D'), 'phrygian':('♮:E','♭:A'), 
                   'lydian':('♮:F','♭:B♭'), 'mixolydian':('♮:G','♭:C'), 'aeolian':('♮:A','♭:D')}
    modal_pairs_reversed = {'ionian':('♭:F','♮:C'), 'dorian':('♮:D','♭:G'), 'phrygian':('♭:A','♮:E'), 
                   'lydian':('♭:B♭','♮:F'), 'mixolydian':('♭:C','♮:G'), 'aeolian':('♭:D','♮:A')}
    major_modes = ('♮:C','♭:F', '♮:F','♭:B♭', '♮:G','♭:C', 
                   'CM', 'GM','DM','AM','EM','BM', 'F♯M','FM','B♭M','E♭M','A♭M','D♭M','G♭M')
    minor_modes = ('♭:G', '♮:D', '♮:E','♭:A', '♮:A','♭:D', 
                   'am','em','bm','f♯m','c♯m','g♯m','d♯m','dm','gm','cm','fm','b♭m','e♭m')
    temp_letters = ('M', 'm')
    tonal_keys = {}
    for a in modal_pairs.values():
        if a in G.edges():
            b = G.get_edge(a[0], a[1])
            b.attr['penwidth']=15
            b.attr['style']='bold'

    for a in modal_pairs_reversed.values():
        if a in G.edges():
            b = G.get_edge(a[0], a[1])
            b.attr['penwidth']=15
            b.attr['style']='bold'

    for x in major_modes:
        if x in G.nodes():
            b = G.get_edge('M', x)
            b.attr['color']='blue'
            b.attr['penwidth']=5
            b.attr['style']='dashed'
    for x in minor_modes:
        if x in G.nodes():
            b = G.get_edge('m', x)
            b.attr['color']='green'
            b.attr['penwidth']=5
            b.attr['style']='dashed'
    for node in G.nodes():
#         print(node)
        if node not in modal_pairs.values() and node not in major_modes and \
            node not in minor_modes and node not in temp_letters:
            G.remove_node(node)
            print('removed:', node)
    M = G.get_node('M')

    M.attr['shape']='square'
    M.attr['fontsize']=100
    m = G.get_node('m')

    m.attr['shape']='square'
    m.attr['fontsize']=100
    G.draw('/home/daniel/Desktop/alf_fq_neato.dot', format='dot', prog='neato')
    G.draw

In [90]:
def neato_graph_big(list_of_lists, label_lists):
    dt = [('len', float)]

    A = (np.array(list_of_lists)).view(dt)

    G = nx.from_numpy_matrix(A)
    relabeled_nodes = {}
    for x, y in zip (range(len(label_lists)), label_lists):
        relabeled_nodes[x] = y[1:-1]
        print(y[1:-1])
    G = nx.relabel_nodes(G, relabeled_nodes)

    G = to_agraph(G)

    G.node_attr.update(style='filled', fontsize=50)
#     G.edge_attr.update(color="black", width=0.5)
    G.edge_attr.update(color='black', style='invis', width=0)
    # G.graph_attr.update(landscape='true', ranksep='0.1')
    #  ♭ ♮ ...for copy and paste
    
    modal_pairs = {'ionian':('♮:C','♭:F'), 'dorian':('♭:G', '♮:D'), 'phrygian':('♮:E','♭:A'), 
                   'lydian':('♮:F','♭:B♭'), 'mixolydian':('♮:G','♭:C'), 'aeolian':('♮:A','♭:D')}
    modal_pairs_reversed = {'ionian':('♭:F','♮:C'), 'dorian':('♮:D','♭:G'), 'phrygian':('♭:A','♮:E'), 
                   'lydian':('♭:B♭','♮:F'), 'mixolydian':('♭:C','♮:G'), 'aeolian':('♭:D','♮:A')}
    major_modes = ('♮:C','♭:F', '♮:F','♭:B♭', '♮:G','♭:C', 
                   'CM', 'GM','DM','AM','EM','BM', 'F♯M','FM','B♭M','E♭M','A♭M','D♭M','G♭M')
    minor_modes = ('♭:G', '♮:D', '♮:E','♭:A', '♮:A','♭:D', 
                   'am','em','bm','f♯m','c♯m','g♯m','d♯m','dm','gm','cm','fm','b♭m','e♭m')
    temp_letters = ('M', 'm')
    tonal_keys = {}
    for a in modal_pairs.values():
        for x in G.edges():
#             print('edge', x)
            if (x[0][2:], x[1][2:]) == a or x == a:
                b = x
                b.attr['penwidth']=15
                b.attr['style']='bold'

    for a in modal_pairs_reversed.values():
        for x in G.edges():
            if (x[0][2:], x[1][2:]) == a or x == a:
                b = x
                b.attr['penwidth']=15
                b.attr['style']='bold'

    for a in major_modes:
        for x in G.nodes():
            if x[2:] == a or x == a:
                print('yes, major')
                b = G.get_edge('M', x)
                b.attr['color']='blue'
                b.attr['penwidth']=5
                b.attr['style']='dashed'
    for a in minor_modes:
        for x in G.nodes():
            if x[2:] == a or x == a:
                print('yes, minor')
                b = G.get_edge('m', x)
                b.attr['color']='green'
                b.attr['penwidth']=5
                b.attr['style']='dashed'
    for node in G.nodes():
#         print(node)
        if node[2:] not in modal_pairs.values() and node[2:] not in major_modes and \
            node[2:] not in minor_modes and node not in temp_letters:
            G.remove_node(node)
            print('removed:', node)
    M = G.get_node('M')

    M.attr['shape']='square'
    M.attr['fontsize']=100
    m = G.get_node('m')

    m.attr['shape']='square'
    m.attr['fontsize']=100
    G.draw('/home/daniel/Desktop/all_corpora_neato.dot', format='dot', prog='neato')
    G.draw

In [8]:
def neato_maker(lists_tuple):
    DIST = []
    for x in lists_tuple[0]:
        dist_temp = []
        for y in lists_tuple[0]:
            dist_temp.append(euclidean(x, y))
        DIST.append(dist_temp)
    return DIST, label_maker(lists_tuple[1])      
        

In [15]:
# X = neato_maker(all_all)
X = neato_maker(alf_ave)

In [18]:
neato_graph(X[0], alf_ave[1])

♮:G
♭:G
♮:C
♮:A
♭:D
♭:F
♮:D
♭:C
♮:F
♭:B♭
♮:E
M
m


In [17]:
X[1]

[]

In [50]:
for x in range(2, 13):
    print(x, k_means_simple(alfabeto_notes_data[0], x, label_maker_alfabeto(alfabeto_notes_data[1]))['completeness'])

2 0.87271084821
3 0.645738074491
4 0.538875095929
5 0.484580596602
6 0.443762877666
7 0.393770280906
8 0.379438935762
9 0.369058269995
10 0.346963226932
11 0.338507285522
12 0.331769042999
